In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

True

QUESTION 1

In [ ]:
from keras.datasets import fashion_mnist
import matplotlib.pyplot as plt
import numpy as np

# dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# class names
class_names = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
]
# Initializing wandb run
wandb.init(project="wandb", name="sample images for each class")

# Finding one sample per class
sample_images = {}
for i in range(len(y_train)):
    label = y_train[i]
    if label not in sample_images:
        sample_images[label] = x_train[i]
    if len(sample_images) == 10:
        break

# Log images to wandb
wandb.log({"sample images for each class": [wandb.Image(sample_images[label], caption=class_names[label]) for label in sample_images]})

# End wandb run
wandb.finish()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


FORWARD PROPAGATION IMPLEMENTATION


In [ ]:
import numpy as np

# Activation Functions
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def tanh(z):
    return np.tanh(z)

def relu(z):
    return np.maximum(0, z)

def softmax(x):
    e_x = np.exp(x - np.max(x, axis=0, keepdims=True))  # Numerical stability
    return e_x / np.sum(e_x, axis=0, keepdims=True)

# Weight Initialization Function
def initialize(initializer, size1, size2):
    if initializer == "random":
        return np.random.randn(size1, size2) * 0.01
    elif initializer == "Xavier":
        return np.random.randn(size1, size2) * np.sqrt(1 / size2)
    else:
        raise ValueError("Invalid initializer. Use 'random' or 'Xavier'.")

# Linear Forward Function
def linear_forward(H, W, b):
    A = np.dot(W, H) + b.reshape(-1, 1)  # Ensure bias broadcasting
    cache = (H, W, b)
    return A, cache

# Initializing weight and bias matrices for all layers
def initialize_Wb_matrix(X, num_hidden, size_hidden, initializer):
    layer_dims = [X.shape[0]] + [size_hidden] * num_hidden + [10]  # Output layer (10 classes)
    np.random.seed(3)

    Wb_matrix = {}
    update = {}
    grads = {}

    for l in range(1, len(layer_dims)):
        Wb_matrix[f'W{l}'] = initialize(initializer, layer_dims[l], layer_dims[l-1])
        update[f'W{l}'] = np.zeros((layer_dims[l], layer_dims[l-1]))
        Wb_matrix[f'b{l}'] = np.zeros((layer_dims[l], 1))  # Ensure column vector
        update[f'b{l}'] = np.zeros((layer_dims[l], 1))
        grads[f'dW{l}'] = np.zeros((layer_dims[l], layer_dims[l-1]))
        grads[f'db{l}'] = np.zeros((layer_dims[l], 1))

    return Wb_matrix, update, grads

# Activation Forward
def activation_forward(H_prev, W, b, activation):
    A, linear_cache = linear_forward(H_prev, W, b)

    if activation == 'relu':
        H = relu(A)
    elif activation == 'sigmoid':
        H = sigmoid(A)
    elif activation == 'tanh':
        H = tanh(A)
    elif activation == 'softmax':
        H = softmax(A)
    else:
        raise ValueError("Invalid activation function.")

    return H, A, linear_cache  # A (pre-activation) added for cache

# Forward Propagation
def forward_propagation(X, Wb_matrix, hidden_activation='relu'):
    H = X
    L = len(Wb_matrix) // 2
    A_caches = []
    H_caches = [H]

    # Hidden layers
    for l in range(1, L):
        H, A_cache, linear_cache = activation_forward(H, Wb_matrix[f'W{l}'], Wb_matrix[f'b{l}'], hidden_activation)
        A_caches.append(A_cache)
        H_caches.append(H)

    # Output layer (Softmax)
    HL, AL, linear_cache = activation_forward(H, Wb_matrix[f'W{L}'], Wb_matrix[f'b{L}'], 'softmax')
    A_caches.append(AL)
    H_caches.append(HL)

    return HL, H_caches, A_caches

BACKWARD PROPAGATION IMPLEMENTATION

In [ ]:
import numpy as np
def one_hot_encoding(l, L):
    #Converts a class index into a one-hot encoded vector.
    e = [1 if i == l else 0 for i in range(L)]
    return np.asarray(e)
def deriv_activ(z, activation):
    #Computes the derivative of the activation function.
  if(activation == "relu"):
    z[z<=0] = 0
    z[z>0] = 1
    return z
  elif(activation == "sigmoid"):
    g_deriv = sigmoid(z) * (1 - sigmoid(z))
    return g_deriv

  elif(activation == "tanh"):
    deriv = 1 - (tanh(z)) ** 2
    return deriv
def back_propagation(H_caches, A_caches, Wb_matrix, y_hat, y, activation, weight_decay,loss):
    #performs backpropagation to compute gradients
  e_y = []
  L = int((len(Wb_matrix)/2))
  num_train = y_hat.shape[1]
  num_classes = y_hat.shape[0]
  for j in range(num_train):
    e_y.append(one_hot_encoding(y[j],num_classes))
  e_y = np.reshape(e_y,(num_classes,num_train))
  if loss == "cross_entropy":
    da = -(e_y - y_hat)
  elif loss == "mse":
    da = (y_hat - e_y)*y_hat - y_hat*(np.dot((y_hat-y).T, y_hat))
  da = -(e_y - y_hat)
  grads = {}

  for k in reversed(range(1,L+1)):
    dW = np.matmul(da, np.transpose(H_caches[k-1])) + 2*weight_decay*Wb_matrix['W{:d}'.format(k)]
    db = da
    if k != 1:
      dh = np.dot(Wb_matrix['W{:d}'.format(k)].T, da)
      da = np.multiply(dh, deriv_activ(A_caches[k-2], activation))
    grads["dW" + str(k)] = dW
    grads["db" + str(k)] = db

  return grads
def sgd(Wb_matrix, grads, eta,L):
    #Performs Stochastic Gradient Descent (SGD) optimization.
  for l in range(L):
    Wb_matrix['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - eta * grads["dW" + str(l+1)]
    Wb_matrix['b' + str(l+1)] = Wb_matrix['b' + str(l+1)] - eta * grads["db" + str(l+1)]
  return Wb_matrix

def momentum(Wb_matrix, grads, eta, gamma, update,L):
    #Performs Momentum-based optimization.
  for l in range(L):
    update['W' + str(l+1)] = (gamma * update['W' + str(l+1)]) + eta * grads['dW' + str(l+1)]
    Wb_matrix['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - update['W' + str(l+1)]

    update["b" + str(l+1)] = (gamma * update["b" + str(l+1)]) + eta * grads["db" + str(l+1)]
    Wb_matrix["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - update["b" + str(l+1)]

  return Wb_matrix, update

def nag(Wb_matrix,eta,gamma, update, L):
    #Performs Nesterov Accelerated Gradient (NAG) optimization.
  Wb_matrix_temp= {}
  for l in range(L):
    update['W' + str(l+1)] = (gamma * update['W' + str(l+1)])
    Wb_matrix_temp['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - update['W' + str(l+1)]

    update["b" + str(l+1)] = (gamma * update["b" + str(l+1)])
    Wb_matrix_temp["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - update["b" + str(l+1)]

  return Wb_matrix_temp, update

def rmsprop(Wb_matrix, eta, update ,beta1, eps, grads, L):
    #Performs RMS-prop optimization
  for l in range(L):
    update['W' + str(l+1)] = beta1 * update['W' + str(l+1)] + (1-beta1) * grads['dW' + str(l+1)]**2
    update["b" + str(l+1)] = beta1 * update["b" + str(l+1)] + (1-beta1) * grads["db" + str(l+1)]**2
    Wb_matrix['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - (eta/np.sqrt(update['W' + str(l+1)]+eps)) * grads['dW' + str(l+1)]
    Wb_matrix["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - (eta/np.sqrt(update['b' + str(l+1)]+eps)) * grads['db' + str(l+1)]
  return Wb_matrix, update

def adam(Wb_matrix, eta, beta1, beta2, update, v_n, eps, grads, L,i):
  #Performs ADAM optimization
  import math
  m_hat = update.copy()
  v_hat = update.copy()
  for l in range(L):
    update['W' + str(l+1)] = beta1 * update['W' + str(l+1)] + (1-beta1) * grads['dW' + str(l+1)]
    update['b' + str(l+1)] = beta1 * update['b' + str(l+1)] + (1-beta1) * grads['db' + str(l+1)]

    v_n['W' + str(l+1)] = beta2 * v_n['W' + str(l+1)] + (1-beta2) * grads['dW' + str(l+1)]**2
    v_n["b" + str(l+1)] = beta2 * v_n["b" + str(l+1)] + (1-beta2) * grads['db' + str(l+1)]**2

    m_hat['W' + str(l+1)] = update['W' + str(l+1)]/(1-math.pow(beta1,i+1))
    m_hat['b' + str(l+1)] = update['b' + str(l+1)]/(1-math.pow(beta1,i+1))

    v_hat['W' + str(l+1)] = v_n['W' + str(l+1)]/(1-math.pow(beta2,i+1))
    v_hat['b' + str(l+1)] = v_n['b' + str(l+1)]/(1-math.pow(beta2,i+1))


    Wb_matrix['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - (eta/(np.sqrt(v_hat['W' + str(l+1)]+eps)))* m_hat['W' + str(l+1)]
    Wb_matrix["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - (eta/(np.sqrt(v_hat['b' + str(l+1)]+eps)))* m_hat['b' + str(l+1)]


  return Wb_matrix, update, v_n

In [ ]:
def swp():
    from keras.datasets import fashion_mnist
    import numpy as np
    import math
    import random
    from sklearn.metrics import confusion_matrix
    import seaborn as sns
    from sklearn.model_selection import train_test_split
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    X_train_flatten = np.reshape(X_train, (X_train.shape[0], X_train.shape[1] * X_train.shape[2])) / 255.0
    X_test_flatten = np.reshape(X_test, (X_test.shape[0], X_test.shape[1] * X_test.shape[2])) / 255.0
    X_test_flatten = X_test_flatten.T

    X_train_flatten, X_val, y_train, y_val = train_test_split(X_train_flatten, y_train, test_size=0.1, random_state=42)
    X_train_flatten = X_train_flatten.T
    X_val = X_val.T

    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                   'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

    hyperparameter_defaults = dict(
      num_hidden=random.choice([2, 3, 4, 5]),
      size_hidden=random.choice([32, 64, 128]),
      weight_decay=random.choice([0, 0.0005, 0.5]),
      optimizer=random.choice(["sgd", "momentum", "adam", "rmsprop", "nag"]),
      initializer=random.choice(["Xavier", "random"]),
      activation=random.choice(["sigmoid", "relu", "tanh"]),
      batch_size=random.choice([32, 64]),
      learning_rate=random.choice([0.001, 0.01, 0.1]),
      max_epoch=random.choice([5, 10]),
      loss_func=random.choice(["mse", "cross_entropy"]),
)
    wandb.init(project="wandb", config=hyperparameter_defaults)
    config = wandb.config
    wandb.run.name = f"{config.loss_func}_hl{config.num_hidden}_bs_{config.batch_size}_ac_{config.activation}"

    L = config.num_hidden + 1
    beta1 = 0.9
    beta2 = 0.999
    eps = 1e-8
    gamma = 0.9

    Wb_matrix, update, grad_initial = initialize_Wb_matrix(X_train_flatten, config.num_hidden, config.size_hidden, config.initializer)
    v = update.copy()

    for i in range(config.max_epoch):
        loss, val_loss, count = 0, 0, 0
        grads = grad_initial
        print("Epoch:", i + 1)

        for j in range(X_train_flatten.shape[1]):
            train_ip = np.reshape(X_train_flatten[:, j], (-1, 1))
            y_hat, H_caches, A_caches = forward_propagation(train_ip, Wb_matrix, config.activation)

            if y_train[j] == np.argmax(y_hat):
                count += 1

            grad = back_propagation(H_caches, A_caches, Wb_matrix, y_hat, [y_train[j]], config.activation, config.weight_decay, config.loss_func)

            for l in range(L):
                grads['dW' + str(l + 1)] += grad['dW' + str(l + 1)]
                grads['db' + str(l + 1)] += grad['db' + str(l + 1)]

            if (j + 1) % config.batch_size == 0:
                for l in range(L):
                    grads['dW' + str(l + 1)] /= config.batch_size
                    grads['db' + str(l + 1)] /= config.batch_size

                if config.optimizer == 'adam':
                    Wb_matrix, update, v = adam(Wb_matrix, config.learning_rate, beta1, beta2, update, v, eps, grads, L, i + 1)
                elif config.optimizer == 'sgd':
                    Wb_matrix = sgd(Wb_matrix, grads, config.learning_rate, L)
                elif config.optimizer == 'momentum':
                    Wb_matrix, update = momentum(Wb_matrix, grads, config.learning_rate, gamma, update, L)
                elif config.optimizer == 'rmsprop':
                    Wb_matrix, update = rmsprop(Wb_matrix, config.learning_rate, update, beta1, eps, grads, L)

        accuracy = 100 * count / X_train_flatten.shape[1]
        print("     Accuracy", accuracy)
        wandb.log({'epoch': i, 'accuracy': accuracy})

    y_hat_test, _, _ = forward_propagation(X_test_flatten, Wb_matrix, config.activation)
    wandb.log({"Confusion_Matrix": wandb.plot.confusion_matrix(
        probs=None,
        y_true=y_test,
        preds=np.argmax(y_hat_test, axis=0),
        class_names=class_names)})

    return Wb_matrix, config, X_test_flatten, y_test, class_names
    wandb.run.finish()

In [ ]:
for _ in range(4):  #done with 70 runs earli
  swp()

Epoch: 1
     Accuracy 67.9962962962963
Epoch: 2
     Accuracy 79.83148148148148
Epoch: 3
     Accuracy 82.54629629629629
Epoch: 4
     Accuracy 83.71481481481482
Epoch: 5
     Accuracy 84.47222222222223


accuracy,▁▆▇██
epoch,▁▃▅▆█
accuracy,84.47222
epoch,4


Epoch: 1
     Accuracy 14.674074074074074
Epoch: 2
     Accuracy 14.674074074074074
Epoch: 3
     Accuracy 14.674074074074074
Epoch: 4
     Accuracy 14.674074074074074
Epoch: 5
     Accuracy 14.674074074074074


accuracy,▁▁▁▁▁
epoch,▁▃▅▆█
accuracy,14.67407
epoch,4


Epoch: 1
     Accuracy 9.916666666666666
Epoch: 2
     Accuracy 9.905555555555555
Epoch: 3
     Accuracy 9.83148148148148
Epoch: 4
     Accuracy 9.824074074074074
Epoch: 5
     Accuracy 9.862962962962962


...

Epoch: 1
     Accuracy 10.012962962962963
Epoch: 2
     Accuracy 9.91851851851852
Epoch: 3
     Accuracy 9.894444444444444
Epoch: 4
     Accuracy 9.937037037037037
Epoch: 5
     Accuracy 9.95
Epoch: 6
     Accuracy 9.942592592592593
Epoch: 7
     Accuracy 9.896296296296295
Epoch: 8
     Accuracy 9.892592592592592
Epoch: 9
     Accuracy 9.907407407407407
Epoch: 10
     Accuracy 9.905555555555555


accuracy,█▃▁▄▄▄▁▁▂▂
epoch,▁▂▃▃▄▅▆▆▇█
accuracy,9.90556
epoch,9


Epoch: 1
     Accuracy 5.627777777777778
Epoch: 2
     Accuracy 5.627777777777778
Epoch: 3
     Accuracy 5.627777777777778
Epoch: 4
     Accuracy 5.627777777777778
Epoch: 5
     Accuracy 5.627777777777778
Epoch: 6
     Accuracy 5.627777777777778
Epoch: 7
     Accuracy 5.627777777777778
Epoch: 8
     Accuracy 5.627777777777778
Epoch: 9
     Accuracy 5.627777777777778
Epoch: 10
     Accuracy 5.627777777777778


accuracy,▁▁▁▁▁▁▁▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
accuracy,5.62778
epoch,9


Epoch: 1
     Accuracy 10.024074074074074
Epoch: 2
     Accuracy 17.564814814814813
Epoch: 3
     Accuracy 67.86851851851851
Epoch: 4
     Accuracy 81.26481481481481
Epoch: 5
     Accuracy 84.18518518518519
Epoch: 6
     Accuracy 85.29814814814814
Epoch: 7
     Accuracy 86.03148148148148
Epoch: 8
     Accuracy 86.5037037037037
Epoch: 9
     Accuracy 86.85
Epoch: 10


In [ ]:
import wandb

sweep_config = {
  "name": "My Sweep",
  "method": "grid",
  "project": "wandb",
  "metric":{
      "name":"val_accuracy",
      "goal":"maximize"
  },
  "parameters": {
        "max_epoch": {
            "values": [10]
        },
        "num_hidden": {
            "values":[5]
        },
        "size_hidden": {
            "values":[128]
        },
        "weight_decay":{
            "values":[0, 0.0005]
        },
        "learning_rate":{
            "values":[0.001]
        },
        "batch_size": {
            "values":[16]
        },
        "optimizer": {
            "values":['adam', 'nadam']
        },
        "initializer": {
            "values":['Xavier']
        },
        "activation":{
            "values": ['relu']
        },
        "loss_func":{
            "values": ['cross_entropy']
        }
    }
}

sweep_id = wandb.sweep(sweep_config)

In [ ]:
import wandb

sweep_config = {
  "name": "Loss_compare",
  "method": "grid",
  "project": "wandb",
  "metric":{
      "name":"val_accuracy",
      "goal":"maximize"
  },
  "parameters": {
        "max_epoch": {
            "values":[10]
        },
        "num_hidden": {
            "values":[5]
        },
        "size_hidden": {
            "values":[64]
        },
        "weight_decay":{
            "values":[0]
        },
        "learning_rate":{
            "values":[0.001]
        },
        "batch_size": {
            "values":[16]
        },
        "optimizer": {
            "values":['adam']
        },
        "initializer": {
            "values":['Xavier']
        },
        "activation":{
            "values": ['relu']
        },
        "loss_func":{
            "values": ['mse']
        }
    }
}

sweep_id = wandb

In [ ]:
sweep_config = {
  "name": "Cross Entropy Loss-Final(5)",
  "metric": {
      "name":"validation_accuracy",
      "goal": "maximize"
  },
  "method": "random",
  "parameters": {
        "neurons_per_layer": {
            "values": [32, 64, 128, 256, 512]
        },
         "num_hidden_layers": {
            "values": [3, 4, 5, 6]
        },
        "activation_function": {
            "values": ["sigmoid", "relu", "tanh","identity"]
        },
        "learning_rate": {
            "values": [0.01, 0.001, 0.0001]
        },
        "init": {
            "values": ["Xavier", "random"]
        },
        "optimizer": {
            "values": ["sgd", "momentum", "nesterov", "adam", "nadam", "rmsprop"]
        },
        "batch_size": {
            "values": [32,64,128,256]
        },
        "L2_lamb": {
            "values": [0, 0.05, 0.5]
        },
        "epochs": {
            "values": [5, 15, 20, 30, 40]
        }
    }
}
sweep_id = wandb

In [ ]:
def mnist_swp():
  from keras.datasets import mnist
  from matplotlib import pyplot
  import numpy as np
  import math
  import random
  from sklearn.metrics import confusion_matrix
  import seaborn as sns
  from sklearn.model_selection import train_test_split
  (X_train, y_train), (X_test, y_test) = mnist.load_data()
  X_train_flatten = np.reshape(X_train,(X_train.shape[0],X_train.shape[1]*X_train.shape[2]))/255.0

  X_test_flatten = np.reshape(X_test,(X_test.shape[0],X_test.shape[1]*X_test.shape[2]))/255.0
  X_test_flatten = X_test_flatten.T

  X_train_flatten, X_val, y_train, y_val = train_test_split( X_train_flatten, y_train, test_size=0.1, random_state=42)
  X_train_flatten = X_train_flatten.T
  X_val = X_val.T
  class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
  num_train = int(0.9*X_train.shape[0])
  num_val = int(0.1*X_train.shape[0])

  hyperparameter_defaults = dict(
      num_hidden = 5,
      size_hidden = 64,
      weight_decay = 0.0005,
      optimizer = 'adam',
      initializer = 'Xavier',
      activation = 'relu',
      batch_size = 16,
      learning_rate = 0.001,
      max_epoch = 10,
      loss_func = 'cross_entropy'
      )


  wandb.init(project="wandb", config=hyperparameter_defaults)
  config = wandb.config
  wandb.run.name = "MNIST_hl{}_bs_{}_ac_{}".format(config.num_hidden,config.batch_size, config.activation)
  L = config.num_hidden+1
  beta1 = 0.9
  beta2 = 0.999
  eps = 1e-8
  gamma = 0.9


  Wb_matrix, update, grad_initial = initialize_Wb_matrix(X_train_flatten, config.num_hidden , config.size_hidden ,config.initializer)
  #initializing parameters in adam optimization to zero dictionary
  v = update.copy()
  m_hat = update.copy()
  v_hat = update.copy()


  for i in range(config.max_epoch):
    loss = 0
    val_loss = 0
    count = 0
    grads = grad_initial
    print("Epoch:", i+1)
    for j in range(num_train):
      train_ip= np.reshape(X_train_flatten[:,j], (-1, 1))
      y_hat, H_caches, A_caches = forward_propagation(train_ip, Wb_matrix ,config.activation)
      if(y_train[j]==np.argmax(y_hat)):
        count = count + 1;

      if (j+1)%config.batch_size == 0  :
        if config.optimizer == 'nag':
          Wb_matrix, update = nag(Wb_matrix, config.learning_rate,gamma, update, L)
        elif config.optimizer == 'nadam':
          Wb_matrix, update = nag(Wb_matrix, config.learning_rate,gamma, update, L)

      grad = back_propagation(H_caches, A_caches, Wb_matrix, y_hat, [y_train[j]], config.activation, config.weight_decay,config.loss_func)
      for l in range(L):
        grads['dW' + str(l+1)] = grads['dW' + str(l+1)] + grad['dW' + str(l+1)]
        grads['db' + str(l+1)] = grads['db' + str(l+1)] + grad['db' + str(l+1)]


      if (j+1)%config.batch_size == 0 :
        for l in range(L):
          grads['dW' + str(l+1)] = grads['dW' + str(l+1)]/config.batch_size
          grads['db' + str(l+1)] = grads['db' + str(l+1)]/config.batch_size
        if config.optimizer == 'sgd':
          Wb_matrix = sgd(Wb_matrix, grads, config.learning_rate,L)
        elif config.optimizer == 'momentum':
          Wb_matrix,update = momentum(Wb_matrix,grads,config.learning_rate,gamma,update,L)
        elif config.optimizer == 'nag':
          Wb_matrix,update = momentum(Wb_matrix,grads,config.learning_rate,gamma,update,L)
        elif config.optimizer == 'rmsprop':
          Wb_matrix, update = rmsprop(Wb_matrix, config.learning_rate, update,  beta1, eps, grads, L)
        elif config.optimizer == 'adam':
          Wb_matrix, update,v = adam(Wb_matrix, config.learning_rate, beta1, beta2, update, v, eps, grads, L, i+1)
        elif config.optimizer == 'nadam':
          Wb_matrix, update,v = adam(Wb_matrix, config.learning_rate, beta1, beta2, update, v, eps, grads, L, i+1)
      if config.loss_func == 'cross_entropy':
        loss = loss - (1/num_train)*math.log(y_hat[y_train[j]])
      elif config.loss_func == 'mse':
        loss = loss + (1/num_train)*(np.argmax(y_hat) - y_train[j])**2

    y_hat_val, H_caches_val, A_caches_val = forward_propagation(X_val, Wb_matrix ,config.activation)
    count_val = np.sum(np.argmax(y_hat_val, axis = 0)== y_val)

    for j in range(num_val):
      val_ip= np.reshape(X_val[:,j], (-1, 1))
      y_hat_val, H_caches_val, A_caches_val = forward_propagation(val_ip, Wb_matrix ,config.activation)
      if config.loss_func == 'cross_entropy':
        val_loss = val_loss - (1/num_val)*math.log(y_hat_val[int(y_val[j])])
      elif config.loss_func == 'mse':
        val_loss = val_loss + (1/num_val)*(np.argmax(y_hat_val) - y_val[j])**2

    accuracy = 100*count/num_train
    val_accuracy = 100*count_val/num_val

    print("     Loss", loss)
    print("     Accuracy",accuracy)
    print("     Validation Loss", val_loss)
    print("     Validation Accuracy", val_accuracy)

    metrics = {'epoch':i, 'val_accuracy': val_accuracy, 'val_loss': val_loss, 'accuracy': accuracy, 'loss': loss}
    wandb.log(metrics)

  y_hat_test, H_caches_test, A_caches_test = forward_propagation(X_test_flatten, Wb_matrix ,config.activation)
  wandb.log({"Confusion_Matrix" : wandb.plot.confusion_matrix(
                        probs=None,
                        y_true=y_test,
                        preds=np.argmax(y_hat_test, axis = 0),
                        class_names=class_names)})


  wandb.run.finish()

In [ ]:
mnist_swp()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Epoch: 1


<ipython-input-36-d4af7b3cc01d>:96: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  loss = loss - (1/num_train)*math.log(y_hat[y_train[j]])
<ipython-input-36-d4af7b3cc01d>:107: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  val_loss = val_loss - (1/num_val)*math.log(y_hat_val[int(y_val[j])])


     Loss 0.316195736911371
     Accuracy 90.37222222222222
     Validation Loss 0.18549879108774947
     Validation Accuracy 94.63333333333334
Epoch: 2
     Loss 0.1581142536184333
     Accuracy 95.33333333333333
     Validation Loss 0.14159889862629507
     Validation Accuracy 95.85
Epoch: 3
     Loss 0.12371416814136577
     Accuracy 96.34814814814816
     Validation Loss 0.12483939227732761
     Validation Accuracy 96.33333333333333
Epoch: 4
     Loss 0.10410452547801637
     Accuracy 96.94814814814815
     Validation Loss 0.11512925328921049
     Validation Accuracy 96.45
Epoch: 5
     Loss 0.09013265717746007
     Accuracy 97.40925925925926
     Validation Loss 0.10770637660469153
     Validation Accuracy 96.58333333333333
Epoch: 6
     Loss 0.07979201315613145
     Accuracy 97.73333333333333
     Validation Loss 0.10147077621831398
     Validation Accuracy 96.71666666666667
Epoch: 7
     Loss 0.07164757708142258
     Accuracy 97.9888888888889
     Validation Loss 0.0984927742199

accuracy,▁▅▆▇▇▇████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▃▂▂▂▁▁▁▁
val_accuracy,▁▅▆▆▇▇▇▇██
val_loss,█▅▃▃▂▂▁▁▁▁
accuracy,98.46111
epoch,9
loss,0.05464
val_accuracy,96.95
val_loss,0.09411
